<h1><span style="color:goldenrod"><b><center>Customer churn project</center></b></span></h1>

<h2><span style="color:coral"><u><center>Importing the libraries</center></u></span></h2>

In [17]:
import numpy as np
import pandas as pd
import re
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

<h2><span style="color:coral"><u><center>Reading the data</center></u></span></h2>

In [2]:
df = pd.read_excel("../data/Telco_customer_churn.xlsx")

In [18]:
df.head(5)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


<h2><span style="color:coral"><u><center>Info of the data</center></u></span></h2>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [5]:
df.describe()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value,Churn Score,CLTV
count,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,1.0,93521.964646,36.282441,-119.798880,32.371149,64.761692,0.265370,58.699418,4400.295755
std,0.0,1865.794555,2.455723,2.157889,24.559481,30.090047,0.441561,21.525131,1183.057152
min,1.0,90001.000000,32.555828,-124.301372,0.000000,18.250000,0.000000,5.000000,2003.000000
25%,1.0,92102.000000,34.030915,-121.815412,9.000000,35.500000,0.000000,40.000000,3469.000000
50%,1.0,93552.000000,36.391777,-119.730885,29.000000,70.350000,0.000000,61.000000,4527.000000
75%,1.0,95351.000000,38.224869,-118.043237,55.000000,89.850000,1.000000,75.000000,5380.500000
max,1.0,96161.000000,41.962127,-114.192901,72.000000,118.750000,1.000000,100.000000,6500.000000


<h2><span style="color:coral"><u><center>Separating columns in different lists based on non-object type and object type</center></u></span></h2>

In [6]:
list_of_cols_object = [col for col in df.columns if df[col].dtype == "object"]
list_of_cols_not_object = [col for col in df.columns if df[col].dtype != "object"]
print(list_of_cols_object)
print(list_of_cols_not_object)

['CustomerID', 'Country', 'State', 'City', 'Lat Long', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Total Charges', 'Churn Label', 'Churn Reason']
['Count', 'Zip Code', 'Latitude', 'Longitude', 'Tenure Months', 'Monthly Charges', 'Churn Value', 'Churn Score', 'CLTV']


<h2><span style="color:coral"><u><center>Checking for null values in all columns</center></u></span></h2>

In [7]:
for col in df.columns:
    if df[col].isnull().sum() > 0:
        print(f"Column name = {col};  total null values = {df[col].isnull().sum()}")

Column name = Churn Reason;  total null values = 5174


<h2><span style="color:coral"><u><center>Replacing NaN with 0</center></u></span></h2>

In [19]:
# display(df["Churn Reason"].unique())
display(df.nunique())

CustomerID           7043
Count                   1
Country                 1
State                   1
City                 1129
Zip Code             1652
Lat Long             1652
Latitude             1652
Longitude            1651
Gender                  2
Senior Citizen          2
Partner                 2
Dependents              2
Tenure Months          73
Phone Service           2
Multiple Lines          3
Internet Service        3
Online Security         3
Online Backup           3
Device Protection       3
Tech Support            3
Streaming TV            3
Streaming Movies        3
Contract                3
Paperless Billing       2
Payment Method          4
Monthly Charges      1585
Total Charges        6531
Churn Label             2
Churn Value             2
Churn Score            85
CLTV                 3438
Churn Reason           21
dtype: int64

In [13]:
# df["Churn Reason"] = df["Churn Reason"].fillna(0)
df = df.fillna(0)


In [14]:
display(df["Churn Reason"].unique())

array(['Competitor made better offer', 'Moved',
       'Competitor had better devices',
       'Competitor offered higher download speeds',
       'Competitor offered more data', 'Price too high',
       'Product dissatisfaction', 'Service dissatisfaction',
       'Lack of self-service on Website', 'Network reliability',
       'Limited range of services',
       'Lack of affordable download/upload speed',
       'Long distance charges', 'Extra data charges', "Don't know",
       'Poor expertise of online support',
       'Poor expertise of phone support', 'Attitude of service provider',
       'Attitude of support person', 'Deceased', 0], dtype=object)

<h2><span style="color:coral"><u><center>Converting object type to respective data types</center></u></span></h2>

In [73]:
string_pattern = re.compile(r'^[\d\w\s,_\-]+$')
int_pattern = re.compile(r'^-?\d+$')
float_pattern = re.compile(r'^-?\d*\.\d+$')
datetime_pattern = re.compile(
        r'^\d{4}-\d{2}-\d{2}( \d{2}:\d{2}(:\d{2}(\.\d{3})?)?)?$'
    )

for i in list_of_cols_object:
    col_values = df[i].dropna().astype(str)
    if col_values.apply(lambda x: bool(string_pattern.match(x))).all():
        df[i] = df[i].astype("string")


In [75]:
for i in df.columns:
    values = df[col].dropna().astype(str).sample(min(100, len(df[col])))
    if values.apply(lambda x: bool(int_pattern.match(x))).all():
        df[i] = pd.to_numeric(df[i], downcast='integer', errors='coerce')
    elif values.apply(lambda x: bool(float_pattern.match(x))).all():
        df[i] = pd.to_numeric(df[i], downcast='float', errors='coerce')
    elif values.apply(lambda x: bool(datetime_pattern.match(x))).all():
        df[i] = pd.to_datetime(df[i], errors='coerce')

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   string 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   string 
 3   State              7043 non-null   string 
 4   City               7043 non-null   string 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   string 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   string 
 10  Senior Citizen     7043 non-null   string 
 11  Partner            7043 non-null   string 
 12  Dependents         7043 non-null   string 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   string 
 15  Multiple Lines     7043 non-null   string 
 16  Internet Service   7043 

In [40]:
df.head(2)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved


In [77]:
df[["Payment Method", "Churn Reason"]] = df[["Payment Method", "Churn Reason"]].astype("string")

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   string 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   string 
 3   State              7043 non-null   string 
 4   City               7043 non-null   string 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   string 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   string 
 10  Senior Citizen     7043 non-null   string 
 11  Partner            7043 non-null   string 
 12  Dependents         7043 non-null   string 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   string 
 15  Multiple Lines     7043 non-null   string 
 16  Internet Service   7043 